In [1]:
import sys
sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/python-utils-ak')
from utils_ak.reflexion import load as _load

def _import_globals(module):
    for x in dir(module):
        if not x.startswith('_'):
            globals()[x] = getattr(module, x)
            
def load(module_obj, module_name=None, reload=False, import_globals=False):
    module = _load(module_obj, module_name, reload, import_globals=False)
    if import_globals:
        _import_globals(module)
    return module

In [12]:
request = {'Моцарелла Фиор ди Латте в воде "Красная птица", 45%, 0,125 кг, ф/п': 1, 
          'Моцарелла Фиор Ди Латте в воде "Pretto", 45%, 0,125 кг, ф/п, (8 шт)': 1}

In [3]:
import pandas as pd

In [4]:
df = pd.DataFrame(request.items(), columns=['name', 'qty'])
df

,name,qty
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1


In [5]:
import sys
sys.path.append('/mnt/6088FEB688FE89B4/Users/arsen/Desktop/code/git/2020.10-umalat/umalat')

In [6]:
from app.models import *
from app.models import Boiling as BoilingModel

In [7]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

sqlite_filepath = '/home/akadaner/Desktop/data.sqlite'
engine = create_engine(f"sqlite:///{sqlite_filepath}")
Session = sessionmaker()
Session.configure(bind=engine)
session = Session()

In [8]:
df['sku'] = df['name'].apply(lambda s: session.query(SKU).filter(SKU.name == s).first())

In [9]:
df

,name,qty,sku
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>


In [10]:
sku = df.iloc[0]['sku']

In [11]:
session.query(BoilingModel).first()

<Boiling 1>

In [12]:
df['boiling'] = df['sku'].apply(lambda sku: session.query(BoilingModel).filter(BoilingModel.id == sku.boiling_id).first())

In [13]:
df

,name,qty,sku,boiling
0,"Моцарелла Фиор ди Латте в воде ""Красная птица""...",1,<SKU 27>,<Boiling 6>
1,"Моцарелла Фиор Ди Латте в воде ""Pretto"", 45%, ...",1,<SKU 24>,<Boiling 6>


In [14]:
lines = ['cheese_maker', 'melting', 'termizator', 'packing']

In [15]:
import interval

def calc_interval_length(i):
    return sum([c[1] - c[0] for c in i])

In [88]:
module = load('2020.11.18 algo.py', import_globals=True, reload=True)

In [89]:
with Block('root', t=10) as root:
    with root.make('boiling', block_num=12) as boiling:
        total_size = 27
        with boiling.make('water_pouring', y=6, t=1) as water_pouring:
            with water_pouring.make(y=0) as upper_block:
                upper_block.make('pouring_label', size=6, block_num=12)
                upper_block.make('pouring_name', size=total_size - 6, name='чильеджина  3,6 альче  8500кг')
            with water_pouring.make(y=1) as lower_block:
                lower_block.make('pouring_and_fermenting', size=8)
                lower_block.make('soldification', size=7)
                lower_block.make('cutting', size=7)
                lower_block.make('pouring_off', size=3)
                lower_block.make('extra', size=total_size - 8 - 8 - 7 - 3)
    
        root.make('termizator', size=6)
    
        with root.make('melting', t=22) as melting:
            total_size = 25
            with melting.make('upper_block') as upper_block:
                upper_block.make('melting_label', size=3)
                upper_block.make('melting_name', size=total_size-3, name='чильеджина 5кг/сердечки/фдл 0,125(безлактозная)/0,1')
            with melting.make('lower_block') as lower_block:
                

root {'t': 10}
  boiling {'block_num': 12, 't': 0}
    water_pouring {'y': 6, 't': 0}
      block {'y': 0, 't': 0}
        pouring {'size': 6, 'block_num': 12, 't': 0}
        pouring_name {'size': 21, 'name': 'чильеджина  3,6 альче  8500кг', 't': 6}
      block {'y': 1, 't': 0}
        pouring_and_fermenting {'size': 8, 't': 0}
        soldification {'size': 7, 't': 8}
        cutting {'size': 7, 't': 15}
        pouring_off {'size': 3, 't': 22}
        extra {'size': 1, 't': 25}

In [90]:
style = {
    'pouring_label': {'text': '{block_num} налив'},
    'pouring_name': {'text': '{name}'},
    'pouring_and_fermenting': {'text': 'налив/внесение закваски', 'color': 'orange'},
    'soldification': {'text': 'схватка', 'color': 'yellow'},
    'cutting': {'text': 'резка/обсушка', 'color': '#92d050'},
    'pouring_off': {'text': 'слив', 'color': 'red'},
}

# Drawing

In [91]:
from matplotlib import colors

def cast_color(obj):
    if isinstance(obj, (tuple, list)):
        if all(isinstance(x, int) for x in obj):
            return '#{:02x}{:02x}{:02x}'.format(*obj)
        elif all(isinstance(x, float) for x in obj):
            return colors.to_hex(obj)
    elif isinstance(obj, str):
        try:
            return cast_color(colors.to_rgb(obj))
        except:
            pass
    raise Exception('Unknown object type')

In [92]:
def draw_block(x, y, w, h, text, colour):
    if not colour:
        colour = cast_color('white') # default white colour
    SHEET.merge_cells(start_row=y, start_column=x, end_row=y + h - 1, end_column=x + w - 1)
    merged_cell = SHEET.cell(row=y, column=x)  
    merged_cell.value = text
    merged_cell.alignment = Alignment(horizontal='center')
    merged_cell.fill = PatternFill("solid", fgColor=colour[1:])
    
def draw_block_series(x, y, h, blocks):
    cur_x = x
    for w, text, colour in blocks:
        draw_block(cur_x, y, w, h, text, colour)
        cur_x += w
        
def draw_two_line_series(x, y, h, blocks1, blocks2):
    assert sum([block[0] for block in blocks1]) == sum([block[0] for block in blocks2])
    draw_block_series(x, y, h, blocks1)
    draw_block_series(x, y + h, h, blocks2)
    
    
def cast_w(duration):
    # one width is five minutes
    return int(duration / 5)

In [93]:
import pandas as pd
from openpyxl import Workbook
from datetime import datetime
import openpyxl as opx
from openpyxl.styles import Alignment
from openpyxl.styles import PatternFill
import json

In [94]:
# WB = opx.load_workbook(r'2020.10.06 pouring_template.xlsx')
# SHEET = WB.worksheets[0]

# draw_block(10, 10, 1, 1, '12 налив', '#008000')

# draw_block_series(8, 10, 1, 
#                   [
#                       [5, '60 налив', ''], 
#                       [27, 'фиор ди латте  3,3 альче 80 юнитов  8000кг/11,5кг галактаза 5200', '']
#                   ])

# draw_two_line_series(10, 11, 1,
#                   [
#                       [5, '60 налив', ''], 
#                       [27, 'фиор ди латте  3,3 альче 80 юнитов  8000кг/11,5кг галактаза 5200', '']
#                   ],
#                   [
#                       [12, 'налив/внесение закваски', '#FFA500'],
#                       [7, 'схватка', '#FFFF00'],
#                       [8, 'резка/обсушка', '#008000'],
#                       [3, 'слив', '#FF0000'],
#                       [2, '', '']
#                   ])

# draw_block(20, 20, cast_w(15), 1, 'a', '#008000')


# WB.save('output.xlsx')

In [95]:
def draw(block):
    if block.children:
        for child in block.children:
            draw(child)
    else:
        props = block.props or {}
        props.update(style.get(block.block_class, {}))
        text = props.get('text', '')
        color = cast_color(props.get('color', 'white'))
        print(block.props, text.format(**props), color)
        draw_block(block.beg, block.props['y'], block.props['size'], 1, text.format(**props), color)

In [96]:
WB = opx.load_workbook(r'2020.10.06 pouring_template.xlsx')
SHEET = WB.worksheets[0]

In [97]:
root['boiling'].props

{'t': 10, 'block_num': 12}

In [98]:
draw(root)

{'t': 10, 'block_num': 12, 'y': 6, 'size': 6, 'text': '{block_num} налив'} 12 налив #ffffff
{'t': 16, 'block_num': 12, 'y': 6, 'size': 21, 'name': 'чильеджина  3,6 альче  8500кг', 'text': '{name}'} чильеджина  3,6 альче  8500кг #ffffff
{'t': 10, 'block_num': 12, 'y': 7, 'size': 8, 'text': 'налив/внесение закваски', 'color': 'orange'} налив/внесение закваски #ffa500
{'t': 18, 'block_num': 12, 'y': 7, 'size': 7, 'text': 'схватка', 'color': 'yellow'} схватка #ffff00
{'t': 25, 'block_num': 12, 'y': 7, 'size': 7, 'text': 'резка/обсушка', 'color': '#92d050'} резка/обсушка #92d050
{'t': 32, 'block_num': 12, 'y': 7, 'size': 3, 'text': 'слив', 'color': 'red'} слив #ff0000
{'t': 35, 'block_num': 12, 'y': 7, 'size': 1}  #ffffff


In [99]:
WB.save('output.xlsx')